### Create Training Data

In [1]:
import cv2
import numpy as np

In [2]:
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

def face_crop(img):
    #Function detects face and returns cropped  face
    #If no face is detected it returns the input image
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face=img[y:y+h,x:x+w]
        
    return cropped_face

In [3]:
cap=cv2.VideoCapture(0)
count=0
#Collect 100 samples of face from webcam

while True:
    ret,frame = cap.read()
    if face_crop(frame) is not None:
        count+=1
        face=cv2.resize(face_crop(frame),(200,200))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        #save file in specified directory
        file_path='faces/user/'+str(count)+'.jpg'
        cv2.imwrite(file_path,face)
        
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('FaceCropper',face)
    
    else:
        print('Face Not Found')
        pass
    
    if cv2.waitKey(1)==13 or count==100:#13 is the enter key
        break
    
cap.release()
cv2.destroyAllWindows()
print('Collecting Samples Complete')

### Train Model

In [5]:
from os import listdir
from os.path import isfile, join
data_path='faces/user/'
files = [file for file in listdir(data_path) if isfile(join(data_path,file))]

In [6]:
files

['1.jpg',
 '10.jpg',
 '100.jpg',
 '11.jpg',
 '12.jpg',
 '13.jpg',
 '14.jpg',
 '15.jpg',
 '16.jpg',
 '17.jpg',
 '18.jpg',
 '19.jpg',
 '2.jpg',
 '20.jpg',
 '21.jpg',
 '22.jpg',
 '23.jpg',
 '24.jpg',
 '25.jpg',
 '26.jpg',
 '27.jpg',
 '28.jpg',
 '29.jpg',
 '3.jpg',
 '30.jpg',
 '31.jpg',
 '32.jpg',
 '33.jpg',
 '34.jpg',
 '35.jpg',
 '36.jpg',
 '37.jpg',
 '38.jpg',
 '39.jpg',
 '4.jpg',
 '40.jpg',
 '41.jpg',
 '42.jpg',
 '43.jpg',
 '44.jpg',
 '45.jpg',
 '46.jpg',
 '47.jpg',
 '48.jpg',
 '49.jpg',
 '5.jpg',
 '50.jpg',
 '51.jpg',
 '52.jpg',
 '53.jpg',
 '54.jpg',
 '55.jpg',
 '56.jpg',
 '57.jpg',
 '58.jpg',
 '59.jpg',
 '6.jpg',
 '60.jpg',
 '61.jpg',
 '62.jpg',
 '63.jpg',
 '64.jpg',
 '65.jpg',
 '66.jpg',
 '67.jpg',
 '68.jpg',
 '69.jpg',
 '7.jpg',
 '70.jpg',
 '71.jpg',
 '72.jpg',
 '73.jpg',
 '74.jpg',
 '75.jpg',
 '76.jpg',
 '77.jpg',
 '78.jpg',
 '79.jpg',
 '8.jpg',
 '80.jpg',
 '81.jpg',
 '82.jpg',
 '83.jpg',
 '84.jpg',
 '85.jpg',
 '86.jpg',
 '87.jpg',
 '88.jpg',
 '89.jpg',
 '9.jpg',
 '90.jpg',
 '91.jp

In [7]:
train_data,labels=[],[]
count=0
for i in files:
    image_path=data_path+i
    images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    train_data.append(np.asarray(images,dtype=np.uint8))
    labels.append(count)
    count+=1

labels=np.asarray(labels,dtype=np.int32)

model=cv2.face.LBPHFaceRecognizer_create()

model.train(np.asarray(train_data),np.asarray(labels))
print('Model Trained Successfully')

Model Trained Successfully


### Test Your Model

In [8]:
def face_detector(img,size=0.5):
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if faces is ():
        return img,[]
    
    for (x,y,h,w) in faces:
        cv2.rectangle(img,(x,x),(x+w,y+h),(0,255,255),2)
        roi=img[y:y+h,x:x+w]
        roi=cv2.resize(roi,(200,200))
        
    return img,roi



In [10]:
cap=cv2.VideoCapture(0)


while True:
    ret,frame = cap.read()
    image,face=face_detector(frame)
    
    try:
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        results=model.predict(face)
        if results[1]<500:
            confidence=int(100*(1-results[1]/400))
            display_string=str(confidence)+'% Confident it is user'
            
        cv2.putText(image,display_string,(100,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,120,255),2)
        
        if confidence>75:
            cv2.putText(image,'UNLOCKED',(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image,'LOCKED',(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow('Face Recognition', image )
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    
    if cv2.waitKey(1)==13 :#13 is the enter key
        break
    
cap.release()
cv2.destroyAllWindows()


(13, 36.54798154819664)
(13, 36.584914734662725)
(8, 37.3244333768146)
(4, 37.34626118108146)
(8, 36.42603224887828)
(11, 37.169099739842)
(11, 36.05412907401514)
(23, 36.353916842216904)
(5, 36.61441078861167)
(23, 36.6072907165921)
(7, 35.36906252100815)
(4, 36.20101207374043)
(11, 36.53237946930015)
(3, 35.640542273805565)
(4, 36.42125702764617)
(8, 35.402280676084665)
(11, 36.17247453913154)
(8, 36.52795923372506)
(13, 37.049217496511446)
(23, 35.94926890954934)
(4, 35.826210221811024)
(23, 36.147757830629516)
(8, 36.398871324002826)
(4, 36.1981692715544)
(7, 35.508416002975984)
(4, 35.706495574560684)
(11, 36.42073066784133)
(8, 36.09161983593625)
(89, 36.206701720126155)
(6, 37.51844541278107)
(7, 37.037991237152745)
(89, 39.830012805022285)
(23, 36.25541522115775)
(4, 37.08188883464203)
(78, 36.866245767173595)
(7, 36.31561547414742)
(23, 35.98801110910635)
(89, 37.19544395088627)
(78, 37.1406496463634)
(12, 36.52197624820906)
(8, 36.77172357419564)
(89, 38.13085745248458)
(8, 3